In [ ]:
# 업비트 자동거래 시스템 - 대화형 테스트
# VSCode에서 .ipynb 파일로 저장하여 사용

# %% [markdown]
# # 업비트 자동거래 시스템 테스트
# VSCode에서 Jupyter Notebook으로 실행

# %% 1. 환경 설정
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# 메인 코드 import
from upbit_trader import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print("✅ 모듈 로드 완료")

# %% 2. 설정 초기화
# 테스트용 설정 (API 키 없이)
class TestConfig:
    def __init__(self):
        self.access_key = ""
        self.secret_key = ""
        self.trading_params = {
            'initial_capital': 1000000,
            'max_position_size': 0.2,
            'commission': 0.0005
        }
        self.risk_params = {
            'stop_loss_pct': 0.05,
            'take_profit_pct': 0.1,
            'max_daily_loss': 0.02,
            'risk_per_trade': 0.02
        }

config = TestConfig()
trader = UpbitTrader(config)
risk_manager = RiskManager(config)
strategy = AdvancedTradingStrategy(trader, risk_manager)

print("✅ 시스템 초기화 완료")

# %% 3. 현재 시장 상황 확인
markets = ["KRW-BTC", "KRW-ETH", "KRW-XRP"]

for market in markets:
    ticker = trader.get_ticker(market)
    if ticker:
        print(f"{market}: {ticker['trade_price']:,.0f} KRW "
              f"(24h: {ticker['signed_change_rate']*100:+.2f}%)")

# %% 4. 데이터 수집
market = "KRW-BTC"
days = 30

print(f"📊 {market} {days}일 데이터 수집 중...")
df = trader.get_historical_data(market, days=days)

if not df.empty:
    print(f"✅ 수집 완료: {len(df)}개 캔들")
    print(f"기간: {df['date'].min()} ~ {df['date'].max()}")
    print(f"현재가: {df['close'].iloc[-1]:,.0f} KRW")
    
    # 데이터 미리보기
    display(df.tail())

# %% 5. 기술적 지표 계산 및 시각화
# 지표 계산
df = strategy.calculate_indicators(df)

# 시각화
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# 가격과 이동평균
ax1 = axes[0]
ax1.plot(df['date'], df['close'], label='Close', color='black', linewidth=1)
ax1.plot(df['date'], df['sma_20'], label='SMA20', color='blue', alpha=0.7)
ax1.plot(df['date'], df['sma_50'], label='SMA50', color='red', alpha=0.7)
ax1.fill_between(df['date'], df['bb_upper'], df['bb_lower'], alpha=0.1, color='gray')
ax1.set_title(f'{market} Price Chart')
ax1.legend()
ax1.grid(True, alpha=0.3)

# RSI
ax2 = axes[1]
ax2.plot(df['date'], df['rsi'], color='purple')
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
ax2.set_title('RSI')
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

# MACD
ax3 = axes[2]
ax3.plot(df['date'], df['macd'], label='MACD', color='blue')
ax3.plot(df['date'], df['macd_signal'], label='Signal', color='red')
ax3.bar(df['date'], df['macd_histogram'], label='Histogram', alpha=0.3)
ax3.set_title('MACD')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Volume
ax4 = axes[3]
ax4.bar(df['date'], df['volume'], alpha=0.5)
ax4.set_title('Volume')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% 6. 현재 시장 분석
latest = df.iloc[-1]

print("📈 현재 시장 분석")
print("=" * 50)
print(f"가격: {latest['close']:,.0f} KRW")
print(f"RSI: {latest['rsi']:.2f}")
print(f"MACD: {'상승' if latest['macd'] > latest['macd_signal'] else '하락'}")

# 트렌드 판단
if latest['sma_20'] > latest['sma_50']:
    print("트렌드: 🟢 상승")
else:
    print("트렌드: 🔴 하락")

# 볼린저밴드 위치
bb_position = (latest['close'] - latest['bb_lower']) / (latest['bb_upper'] - latest['bb_lower'])
print(f"볼린저밴드 위치: {bb_position:.1%}")

if latest['rsi'] > 70:
    print("⚠️ RSI 과매수 구간")
elif latest['rsi'] < 30:
    print("⚠️ RSI 과매도 구간")

# %% 7. 백테스팅 실행
print("🔄 백테스팅 실행 중...")
backtest, df_signals = strategy.backtest_strategy(df, "테스트 전략")

# 거래 내역 확인
if backtest.trades:
    trades_data = []
    for trade in backtest.trades[-10:]:  # 최근 10개
        trades_data.append({
            '진입일': trade.entry_date.date(),
            '청산일': trade.exit_date.date() if trade.exit_date else None,
            '포지션': trade.position_type.value,
            '진입가': f"{trade.entry_price:,.0f}",
            '청산가': f"{trade.exit_price:,.0f}" if trade.exit_price else None,
            'PnL': f"{trade.pnl:+,.0f}" if trade.pnl else None
        })
    
    trades_df = pd.DataFrame(trades_data)
    display(trades_df)

# %% 8. 다양한 설정으로 백테스팅 비교
results = []

# 다양한 파라미터 테스트
test_params = [
    {'stop_loss': 0.03, 'take_profit': 0.06, 'name': '보수적'},
    {'stop_loss': 0.05, 'take_profit': 0.10, 'name': '표준'},
    {'stop_loss': 0.07, 'take_profit': 0.15, 'name': '공격적'},
]

for params in test_params:
    # 리스크 매니저 재설정
    risk_manager.stop_loss_pct = params['stop_loss']
    risk_manager.take_profit_pct = params['take_profit']
    
    # 백테스팅
    backtest_engine = BacktestEngine(config)
    df_test = strategy.calculate_indicators(df.copy())
    df_test = strategy.generate_signals(df_test)
    
    for i in range(len(df_test)):
        date = df_test['date'].iloc[i]
        price = df_test['close'].iloc[i]
        signal = df_test['signal'].iloc[i]
        
        if signal in ['buy', 'sell']:
            backtest_engine.execute_trade(date, price, signal)
    
    metrics = backtest_engine.get_performance_metrics()
    metrics['전략'] = params['name']
    results.append(metrics)

# 결과 비교
results_df = pd.DataFrame(results)
display(results_df[['전략', 'Total Return (%)', 'Win Rate (%)', 
                    'Max Drawdown (%)', 'Sharpe Ratio']])

# %% 9. 최적 파라미터 찾기
import itertools

# 파라미터 그리드
param_grid = {
    'stop_loss': [0.03, 0.05, 0.07],
    'take_profit': [0.06, 0.10, 0.15],
    'rsi_oversold': [25, 30, 35],
    'rsi_overbought': [65, 70, 75]
}

best_return = -float('inf')
best_params = None

print("🔍 최적 파라미터 탐색 중...")

# 간단한 그리드 서치 (시간이 걸릴 수 있음)
for sl, tp in itertools.product(param_grid['stop_loss'], param_grid['take_profit']):
    risk_manager.stop_loss_pct = sl
    risk_manager.take_profit_pct = tp
    
    backtest_engine = BacktestEngine(config)
    # ... 백테스팅 실행 ...
    
    metrics = backtest_engine.get_performance_metrics()
    if metrics.get('Total Return (%)', 0) > best_return:
        best_return = metrics['Total Return (%)']
        best_params = {'stop_loss': sl, 'take_profit': tp}

print(f"✅ 최적 파라미터: {best_params}")
print(f"수익률: {best_return:.2f}%")

# %% 10. 실시간 모니터링 시뮬레이션
import time
from IPython.display import clear_output

print("🎮 실시간 모니터링 시뮬레이션 (10초간)")

for i in range(10):
    clear_output(wait=True)
    
    # 현재가 조회
    ticker = trader.get_ticker(market)
    current_price = ticker['trade_price']
    change_rate = ticker['signed_change_rate'] * 100
    
    print(f"⏰ {datetime.now().strftime('%H:%M:%S')}")
    print(f"📊 {market}")
    print(f"💰 현재가: {current_price:,.0f} KRW")
    print(f"📈 변동률: {change_rate:+.2f}%")
    
    # 가상의 포지션이 있다고 가정
    if i > 5:
        entry_price = current_price * 0.98
        pnl = (current_price - entry_price) / entry_price * 100
        print(f"\n📍 포지션: LONG")
        print(f"진입가: {entry_price:,.0f} KRW")
        print(f"평가손익: {pnl:+.2f}%")
        
        if pnl < -5:
            print("⚠️ 손절매 신호!")
        elif pnl > 10:
            print("✅ 익절매 신호!")
    
    time.sleep(1)

print("\n시뮬레이션 종료")